In [ ]:
%matplotlib inline

In [ ]:
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cf_units
import iris
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import matplotlib as mpl

In [ ]:
mpl.rcParams['image.cmap'] = 'viridis'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
iris.FUTURE.netcdf_promote = True

In [ ]:
f = iris.load('data.nc')
print(f)

In [ ]:
u, v = f[2], f[4]

In [ ]:
wspd = (u**2 + v**2)**0.5
wspd.rename('wind_magnitude')
print(wspd)

In [ ]:
bbox = [0, 40, 30, 50]

In [ ]:
sublon = iris.Constraint(longitude=lambda cell: 120 <= cell <= 160)
sublat = iris.Constraint(latitude=lambda cell: 30 <= cell <= 60)

cube = wspd.extract(sublon & sublat)

print(cube)

In [ ]:
from iris.analysis import trajectory

pnts = [{'longitude': 155, 'latitude': 35}, {'longitude': 125, 'latitude': 55}]

In [ ]:
traj = trajectory.Trajectory(pnts, sample_count=100)

traj

In [ ]:
traj.sampled_points[:5]

In [ ]:
lon = [d['longitude'] for d in traj.sampled_points]
lat = [d['latitude'] for d in traj.sampled_points]

sampled_points = [('longitude', lon),
                  ('latitude', lat)]

section = trajectory.interpolate(cube, sampled_points)

print(section)

In [ ]:
seclon, seclat = section.coord('longitude').points, section.coord('latitude').points

In [ ]:
lon, lat = cube.coord('longitude').points, cube.coord('latitude').points

In [ ]:
print(cube)
print(wspd)

In [ ]:
proj = ccrs.PlateCarree()

In [ ]:
fig = plt.figure(figsize=(30,10))

ax1 = fig.add_subplot(121, projection=proj)
ax1.coastlines()
ax1.contourf(lon, lat, cube.data[0,0,...])
#---------------------------------
gl = ax1.gridlines(crs=ccrs.PlateCarree(), # using the same projection
                  draw_labels=True, # add labels
                  linewidth=2, color='gray', alpha=0.5, linestyle='--') # grid line specs
# Remove labels above and on the right of the map (note that Python allows the double equality)
gl.xlabels_top = gl.ylabels_right = False
# Format the labels using the formatters imported from cartopy
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
#---------------------------------
ax1.plot(seclon, seclat, color='r', linewidth=3)


ax2 = fig.add_subplot(122)
ax2.contourf(np.arange(section.data.shape[2]), section.coord('pressure_level').points, section.data[0,...])
ax2.invert_yaxis()